# HW 2 - Naive Bayes in Hadoop MR
__`MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Fall 2018`__

In the live sessions for week 2 and week 3 you got some practice designing and debugging Hadoop Streaming jobs. In this homework we'll use Hadoop MapReduce to implement your first parallelized machine learning algorithm: Naive Bayes. As you develop your implementation you'll test it on a small dataset that matches the 'Chinese Example' in the _Manning, Raghavan and Shutze_ reading for Week 2. For the main task in this assignment you'll be working with a small subset of the Enron Spam/Ham Corpus. By the end of this assignment you should be able to:
* __... describe__ the Naive Bayes algorithm including both training and inference.
* __... perform__ EDA on a corpus using Hadoop MR.
* __... implement__ parallelized Naive Bayes.
* __... constrast__ partial, unordered and total order sort and their implementations in Hadoop Streaming.
* __... explain__ how smoothing affects the bias and variance of a Multinomial Naive Bayes model.

As always, your work will be graded both on the correctness of your output and on the clarity and design of your code. __Please refer to the `README` for homework submission instructions.__ 

## Notebook Setup
Before starting, run the following cells to confirm your setup.

In [63]:
# imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
# global vars (paths) - ADJUST AS NEEDED
# path to java archive files for the hadoop streaming application on your machine
JAR_FILE = "/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.0.jar"
# hdfs directory where  we'll store files for this assignment
HDFS_DIR = "/user/root/HW2"
# !pwd
HOME_DIR = "/media/notebooks/Assignments/HW2" # FILL IN HERE eg. /media/notebooks/Assignments/HW2

In [3]:
# save path for use in Hadoop jobs (-cmdenv PATH={PATH})
from os import environ
PATH  = environ['PATH']

In [4]:
# data path
ENRON = "data/enronemail_1h.txt"

# Question 1: Hadoop MapReduce Key Takeaways.  

This assignment will be the only one in which you use Hadoop Streaming to implement a distributed algorithm. The key reason we continue to teach Hadoop streaming is because of the way it forces the programmer to think carefully about what is happening under the hood when you parallelize a calculation. This question will briefly highlight some of the most important concepts that you need to understand about Hadoop Streaming and MapReduce before we move on to Spark next week.   

### Q1 Tasks:

* __a) short response:__ What "programming paradigm" is Hadoop MapReduce based on? What are the main ideas of this programming paradigm and how does MapReduce exemplify these ideas?

* __b) short response:__ What is the Hadoop Shuffle? When does it happen? Why is it potentially costly? Describe one specific thing we can we do to mitigate the cost associated with this stage of our Hadoop Streaming jobs.

* __c) short response:__ In Hadoop Streaming why do the input and output record format of a combiner script have to be the same? [__`HINT`__ _what level of combining does the framework guarantee? what is the relationship between the record format your mapper emits and the format your reducer expects to receive?_]

* __d) short response:__ To what extent can you control the level of parallelization of your Hadoop Streaming jobs? Please be specific.

* __e) short response:__ What change in the kind of computing resources available prompted the creation of parallel computation frameworks like Hadoop? 

### Q1 Student Answers:

> __a)__ The Hadoop MapReduce is based on the functional programming paradigm. Functional programming is about evaluation of mathematical functions and avoids changing state and mutable data. A key feature is the concept of higher-oder functions that can accept other functions as arguments. 
> Map and Reduce can essentially be thought of as higher-oder functions that contains two parameters: a function and a list of variables. The Map/Reduce then apply the specified functions to the lists of variables. 

> __b)__ Hadoop Shuffle is the process by which the system paritions and sorts the mapper results and transfers to the reducers. It's potentially costly because it merges the partition file from each mapprer, combine, and generate a stream of inputs for the reducer. It priotizes the usage of memory but may also use disk (spills) when in-memory circular buffer get full. One way to mitigate such computational cost is to use in-mapper combiners which combines the records with the same key and can greatly reduce the synchronization and communication needed in the shuffle phase. 

> __c)__ Since we can't guarantee that the combiners are always used in a MapReduce framework, which means that reducers will be getting inputs from both the mappers and combiners, we need to make sure that the format of the mapper output (= combiner input) matches that of the combiners.   

> __d)__ You can specify the number of mappers and reducers used in the framework. Increasing the number of tasks increases the framework overhead but increases load balancing and lowers the cost of failures. However, programmers have little control over where/when a mapper/reducer runs. 

> __e)__ The redesign of CPU from single-cole to multi-core to create more power efficient CPU enabled the design of parallel computing. Later on, distributed computing further improve the scalability of parallel computing frameworks like Hadoop.

# Question 2: MapReduce Design Patterns.  

In the last two live sessions and in your readings from Lin & Dyer you encountered a number of techniques for manipulating the logistics of a MapReduce implementation to ensure that the right information is available at the right time and location. In this question we'll review a few of the key techniques you learned.   

### Q2 Tasks:

* __a) short response:__ What are counters (in the context of Hadoop Streaming)? How are they useful? What kinds of counters does Hadoop provide for you? How do you create your own custom counter?

* __b) short response:__ What are composite keys? How are they useful? How are they related to the idea of custom partitioning?

* __c) short response:__ What is the order inversion pattern? What problem does it help solve? How do we implement it? 

### Q2 Student Answers:

> __a)__ Hadoop counters are typically used to monitor how many tasks have been performed at each stage and how many tasks haven't yet been processed. They are useful for users to monitor load balancing when doing Hadoop streaming. Hadoop provides the Job Tracking counters that includes File System counter, Job counter, and so on. You may create custom counter in by prining incremental outputs under certain conditions in the mapper, combiner, or reducer scripts.

> __b)__ Composite keys are keys that are contructed using more than one values. They are useful when we want to not just sort by the intermediate key but also by some other value emitted by the mapper. Secondary sorting can then be performed to support custom partitioning. 

> __c)__ The order inversion pattern customizes the sorting phase of MapReduce to push data needed for calculations to the reducer ahead of the data that will be manipulated. It helps ensure that the spcial key-value pairs representing the partial marginal contributions are processed before the normal key-value pairs, in the context of relative-frequency calculation.

> To implement order inversion, quoting from the Lin & Dyer:

>>"Emitting a special key-value pair for each co-occurring word pair in the mapper
to capture its contribution to the marginal.

>>Controlling the sort order of the intermediate key so that the key-value pairs
representing the marginal contributions are processed by the reducer before any
of the pairs representing the joint word co-occurrence counts.

>>Defining a custom partitioner to ensure that all pairs with the same left word are
shuffled to the same reducer.

>>Preserving state across multiple keys in the reducer to first compute the marginal
based on the special key-value pairs and then dividing the joint counts by the
marginals to arrive at the relative frequencies."


# Question 3: Understanding Total Order Sort

The key challenge in distributed computing is to break a problem into a set of sub-problems that can be performed without communicating with each other. Ideally, we should be able to define an arbirtary number of splits and still get the right result, but that is not always possible. Parallelization becomes particularly challenging when we need to make comparisons between records, for example when sorting. Total Order Sort allows us to order large datasets in a way that enables efficient retrieval of results. Before beginning this assignment, make sure you have read and understand the [Total Order Sort Notebook](https://github.com/UCB-w261/main/blob/master/Resources/TotalSortGuide/_total-sort-guide-spark2.01-JAN27-2017.ipynb). You can skip the first two MRJob sections, but the rest of section III and all of section IV are **very** important (and apply to Hadoop Streaming) so make sure to read them closely. Feel free to read the Spark sections as well but you won't be responsible for that material until later in the course. To verify your understanding, answer the following questions.

### Q3 Tasks:

* __a) short response:__ What is the difference between a partial sort, an unordered total sort, and a total order sort? From the programmer's perspective, what does total order sort allow us to do that we can't with unordered total? Why is this important with large datasets?

* __b) short response:__ Which phase of a MapReduce job is leveraged to implement Total Order Sort? Which default behaviors must be changed. Why must they be changed?

* __c) short response:__ Describe in words how to configure a Hadoop Streaming job for the custom sorting and partitioning that is required for Total Order Sort.  

* __d) short response:__ Explain why we need to use an inverse hash code function.

* __e) short response:__ Where does this function need to be located so that a Total Order Sort can be performed?

### Q3 Student Answers:

> __a)__ A partial sort is a sort on mapper output keys within each partition, but not across partitions. 

>An unordered totoal sort is the first step for a cross-partition sorting where custom partitioning keys are used so that records get sorted into different partitions using a specify order. But due to the hash funciton used in Hadoop, an unordered total sort is when your data is sorted within each parition and across partitions but the partitions are not sorted in the right order. 

>In a total order sort, your data is sorted not only within each partition but the partitions are also in the right order, you've achieved a total order sort.

>From the programmer's perspective, with a total order sort, you will have ordered part files so that your top results will be in the first part file and the bottom results will be in the last. This is super important because when there is a lot of part files, it'll be hard to identify the part file order if they are not ordered. This can add overhead computation for file reading and writing



> __b)__ Shuffle phase is used to implement Total Order Sort. We need to use custom sorting and partitioning. By default, Hadoop sorting only sort keys within a partition. Custom sorting and partitioning allow keys to be sorted across paritions and paritions to also be sorted. 

> __c)__ 

>>1) Include a custom partition function inside mappers so that records are essentially sorted across partitions 

>>2) Drop internal partition key inside reducers and recover original (key,value) pairs 

>>3) Post-processing step to order paritions: typically take one record from each partition output and construct the appropriate ordering among partitions.

> __d)__ By default, Hadoop uses a hash function to compute the partition index for keys produced by mappers. An inverse hashcode function would allow us the take the desired partition index and total number of partitions as inputs, and compute a partition key. This will allow us to properly assign partition key to implement total order sort

> __e)__ This funciton needs to be placed in the partitioner for the Total Order Sort implementation.  

# About the Data
For the main task in this portion of the homework you will train a classifier to determine whether an email represents spam or not. You will train your Naive Bayes model on a 100 record subset of the Enron Spam/Ham corpus available in the HW2 data directory (__`HW2/data/enronemail_1h.txt`__).

__Source:__   
The original data included about 93,000 emails which were made public after the company's collapse. There have been a number raw and preprocessed versions of this corpus (including those available [here](http://www.aueb.gr/users/ion/data/enron-spam/index.html) and [here](http://www.aueb.gr/users/ion/publications.html)). The subset we will use is limited to emails from 6 Enron employees and a number of spam sources. It is part of [this data set](http://www.aueb.gr/users/ion/data/enron-spam/) which was created by researchers working on personlized Bayesian spam filters. Their original publication is [available here](http://www.aueb.gr/users/ion/docs/ceas2006_paper.pdf). __`IMPORTANT!`__ _For this homework please limit your analysis to the 100 email subset which we provide. No need to download or run your analysis on any of the original datasets, those links are merely provided as context._

__Preprocessing:__  
For their work, Metsis et al. (the authors) appeared to have pre-processed the data, not only collapsing all text to lower-case, but additionally separating "words" by spaces, where "words" unfortunately include punctuation. As a concrete example, the sentence:  
>  `Hey Jon, I hope you don't get lost out there this weekend!`  

... would have been reduced by Metsis et al. to the form:  
> `hey jon , i hope you don ' t get lost out there this weekend !` 

... so we have reverted the data back toward its original state, removing spaces so that our sample sentence would now look like:
> `hey jon, i hope you don't get lost out there this weekend!`  

Thus we have at least preserved contractions and other higher-order lexical forms. However, one must be aware that this reversion is not complete, and that some object (specifically web sites) will be ill-formatted, and that all text is still lower-cased.


__Format:__   
All messages are collated to a tab-delimited format:  

>    `ID \t SPAM \t SUBJECT \t CONTENT \n`  

where:  
>    `ID = string; unique message identifier`  
    `SPAM = binary; with 1 indicating a spam message`  
    `SUBJECT = string; title of the message`  
    `CONTENT = string; content of the message`   
    
Note that either of `SUBJECT` or `CONTENT` may be "NA", and that all tab (\t) and newline (\n) characters have been removed from both of the `SUBJECT` and `CONTENT` columns.  

In [5]:
# take a look at the first 100 characters of the first 5 records (RUN THIS CELL AS IS)
!head -n 5 {ENRON} | cut -c-100

0001.1999-12-10.farmer	0	 christmas tree farm pictures	NA
0001.1999-12-10.kaminski	0	 re: rankings	 thank you.
0001.2000-01-17.beck	0	 leadership development pilot	" sally:  what timing, ask and you shall receiv
0001.2000-06-06.lokay	0	" key dates and impact of upcoming sap implementation over the next few week
0001.2001-02-07.kitchen	0	 key hr issues going forward	 a) year end reviews-report needs generating 


In [6]:
# see how many messages/lines are in the file 
#(this number may be off by 1 if the last line doesn't end with a newline)
!wc -l {ENRON}

100 data/enronemail_1h.txt


In [5]:
# make the HDFS directory if it doesn't already exist
!hdfs dfs -mkdir {HDFS_DIR}

In [6]:
# load the data into HDFS (RUN THIS CELL AS IS)
!hdfs dfs -copyFromLocal {ENRON} {HDFS_DIR}/enron.txt

# Question 4:  Enron Ham/Spam EDA.
Before building our classifier, lets get aquainted with our data. In particular, we're interested in which words occur more in spam emails than in real emails. In this question you'll implement two Hadoop MapReduce jobs to count and sort word occurrences by document class. You'll also learn about two new Hadoop streaming parameters that will allow you to control how the records output of your mappers are partitioned for reducing on separate nodes. 

__`IMPORTANT NOTE:`__ For this question and all subsequent items, you should include both the subject and the body of the email in your analysis (i.e. concatetate them to get the 'text' of the document).

### Q4 Tasks:
* __a) code:__ Complete the missing components of the code in __`EnronEDA/mapper.py`__ and __`EnronEDA/reducer.py`__ to create a Hadoop MapReduce job that counts how many times each word in the corpus occurs in an email for each class. Pay close attention to the data format specified in the docstrings of these scripts _-- there are a number of ways to accomplish this task, we've chosen this format to help illustrate a technique in `part e`_. Run the provided unit tests to confirm that your code works as expected then run the provided Hadoop streaming command to apply your analysis to the Enron data.


* __b) code + short response:__ How many times does the word "__assistance__" occur in each class? (`HINT:` Use a `grep` command to read from the results file you generated in '`a`' and then report the answer in the space provided.)


* __c) short response:__ Would it have been possible to add some sorting parameters to the Hadoop streaming command that would cause our `part a` results to be sorted by count? Briefly explain why or why not.


* __d) code + short response:__ Write a second Hadoop MapReduce job to sort the output of `part a` first by class and then by count. Run your job and save the results to a local file using the provide code. Then describe in words how you would go about printing the top 10 words in each class given this sorted output. (`HINT 1:` _remember that you can simply pass the `part a` output directory to the input field of this job; `HINT 2:` since this task is just reodering the records from `part a` we don't need to write a mapper or reducer, just use `/bin/cat` for both_)


* __ e) code:__ A more efficient alternative to '`grep`-ing' for the top 10 words in each class would be to use the Hadoop framework to separate records from each class into its own partition so that we can just read the top lines in each. Edit your job from ` part d` to specify 2 reduce tasks and to tell Hadoop to partition based on the second field (which indicates spam/ham in our data). Your code should maintain the secondary sort -- that is each partition should list words from most to least frequent.

### Q4 Student Answers:
> __b)__ The word "assistance" occured 8 times in spam emails and 2 times in normal emails

> __c)__ No; since the stream of word has output word + class + 1 and the count hasn't been combined, there isn't a way for us to sort the words by their aggregated counts.   

> __d)__ I would find and print the first ten lines of the two sorted partition files  

In [9]:
# part a - do your work in the provided scripts then RUN THIS CELL AS IS
!chmod a+x EnronEDA/mapper.py
!chmod a+x EnronEDA/reducer.py

In [10]:
# part a - unit test EnronEDA/mapper.py (RUN THIS CELL AS IS)
!echo -e "d1	1	title	body\nd2	0	title	body" | EnronEDA/mapper.py

title	1	1
body	1	1
title	0	1
body	0	1


In [11]:
# part a - unit test EnronEDA/reducer.py (RUN THIS CELL AS IS)
!echo -e "one	1	1\none	0	1\none	0	1\ntwo	0	1" | EnronEDA/reducer.py

one	1	1
one	0	2
two	0	1


In [12]:
# part a - clear output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-output

rm: `/user/root/HW2/eda-output': No such file or directory


In [13]:
# part a - Hadoop streaming job (RUN THIS CELL AS IS)
!hadoop jar {JAR_FILE} \
  -files EnronEDA/reducer.py,EnronEDA/mapper.py \
  -mapper mapper.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/enron.txt \
  -output {HDFS_DIR}/eda-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.0.jar] /tmp/streamjob7151458004723333486.jar tmpDir=null
18/09/25 20:31:22 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/09/25 20:31:22 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/09/25 20:31:23 INFO mapred.FileInputFormat: Total input paths to process : 1
18/09/25 20:31:23 INFO mapreduce.JobSubmitter: number of splits:2
18/09/25 20:31:23 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1537907394596_0001
18/09/25 20:31:24 INFO impl.YarnClientImpl: Submitted application application_1537907394596_0001
18/09/25 20:31:24 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1537907394596_0001/
18/09/25 20:31:24 INFO mapreduce.Job: Running job: job_1537907394596_0001
18/09/25 20:31:32 INFO mapreduce.Job: Job job_1537907394596_0001 running in uber mode : false
18/09/25 20:31:32 INFO mapreduce.Job:  map 0% reduce 0%
18/09

In [14]:
# part a - retrieve results from HDFS & copy them into a local file (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/eda-output/part-0000* > EnronEDA/results.txt

In [15]:
# part b - write your grep command here
!grep assistance  EnronEDA/results.txt

assistance	1	8
assistance	0	2


In [48]:
# part d/e - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-sort-output

Deleted /user/root/HW2/eda-sort-output


In [49]:
# part d/e - write your Hadoop streaming job here
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keypartitioner.options="-k2,2" \
  -D mapreduce.partition.keycomparator.options="-K2,2 -k3,3nr" \
  -mapper /bin/cat \
  -reducer /bin/cat \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -input {HDFS_DIR}/eda-output \
  -output {HDFS_DIR}/eda-sort-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.0.jar] /tmp/streamjob4163844766896019299.jar tmpDir=null
18/09/25 20:57:44 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/09/25 20:57:45 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/09/25 20:57:45 INFO mapred.FileInputFormat: Total input paths to process : 2
18/09/25 20:57:45 INFO mapreduce.JobSubmitter: number of splits:2
18/09/25 20:57:45 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1537907394596_0011
18/09/25 20:57:45 INFO impl.YarnClientImpl: Submitted application application_1537907394596_0011
18/09/25 20:57:46 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1537907394596_0011/
18/09/25 20:57:46 INFO mapreduce.Job: Running job: job_1537907394596_0011
18/09/25 20:57:53 INFO mapreduce.Job: Job job_1537907394596_0011 running in uber mode : false
18/09/25 20:57:53 INFO mapreduce.Job:  map 0% reduce 0%
18/09

In [50]:
# part e - view the top 10 records from each partition (RUN THIS CELL AS IS)
for idx in range(2):
    print(f"\n===== part-0000{idx}=====\n")
    !hdfs dfs -cat {HDFS_DIR}/eda-sort-output/part-0000{idx} | head


===== part-00000=====

the	0	549	
to	0	398	
ect	0	382	
and	0	278	
of	0	230	
hou	0	206	
a	0	196	
in	0	182	
for	0	170	
on	0	135	
cat: Unable to write to output stream.

===== part-00001=====

the	1	698	
to	1	566	
and	1	392	
your	1	357	
a	1	347	
you	1	345	
of	1	336	
in	1	236	
for	1	204	
com	1	153	
cat: Unable to write to output stream.


__Expected output:__
<table>
<th>part-00000:</th>
<th>part-00001:</th>
<tr><td><pre>
the	0	549	
to	0	398	
ect	0	382	
and	0	278	
of	0	230	
hou	0	206	
a	0	196	
in	0	182	
for	0	170	
on	0	135
</pre></td>
<td><pre>
the	1	698	
to	1	566	
and	1	392	
your	1	357	
a	1	347	
you	1	345	
of	1	336	
in	1	236	
for	1	204	
com	1	153
</pre></td></tr>
</table>

# Question 5: Counters and Combiners.
Tuning the number of mappers & reducers is helpful to optimize very large distributed computations. Doing so successfully requires a thorough understanding of the data size at each stage of the job. As you learned in the week3 live session, counters are an invaluable resource for understanding this kind of detail. In this question, we will take the EDA performed in Question 4 as an opportunity to illustrate some related concepts.

### Q5 Tasks:
* __a) short response:__ Read the Hadoop output from your job in Question 4a to report how many records are emitted by the mappers and how many records are received be the reducers. In the context of word counting what does this number represent practically?

* __b) code:__ Note that we wrote the reducer in question 4a such that the input and output record format is identical. This makes it easy to use the same reducer script as a combiner. In the space provided below, write the Hadoop Streaming command to re-run your job from question 4a with this combining added.

* __c) short response__: Report the number of records emitted by your mappers in part b and the number of records received by your reducers. Compare your results here to what you saw in part a. Explain.

* __d) short response__: Describe a scenario where using a combiner would _NOT_ improve the efficiency of the shuffle stage. Explain. [__`BONUS:`__ how does increasing the number of mappers affect the usefulness of a combiner?]

### Q5 Student Answers:
> __a)__ According to the job output, 31490 records are emitted by the mappers and the same amount of records are received by the reducers. This number represent the total number of word occurrences in the 100 emails.

> __c)__ The number of records emitted by the mappers is still 31490 but the number of records received by the reducers decreased to 7638. That's because the implemented combiners have done mapper-level aggregation before feeding records into reducers.   

> __d)__ If the input to the mappers are extremely sparse (worst case: a corpus with maximum word occurance of 1), then the combiner would not increase the efficiency of the shuffle stage since the output from the combiners will be the same as the outputs from the mappers. The more mappers we use, the less useful the combiners will be since the results from each mapper would be more sparse.  

In [57]:
!wc -l EnronEDA/results.txt

6060 EnronEDA/results.txt


In [58]:
# part b - clear output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-output

Deleted /user/root/HW2/eda-output


In [59]:
# part b - write your Hadoop streaming job here
!hadoop jar {JAR_FILE} \
  -files EnronEDA/reducer.py,EnronEDA/mapper.py \
  -mapper mapper.py \
  -combiner reducer.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/enron.txt \
  -output {HDFS_DIR}/eda-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.0.jar] /tmp/streamjob4589378496443444148.jar tmpDir=null
18/09/25 21:45:48 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/09/25 21:45:49 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/09/25 21:45:50 INFO mapred.FileInputFormat: Total input paths to process : 1
18/09/25 21:45:50 INFO mapreduce.JobSubmitter: number of splits:2
18/09/25 21:45:50 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1537907394596_0012
18/09/25 21:45:50 INFO impl.YarnClientImpl: Submitted application application_1537907394596_0012
18/09/25 21:45:50 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1537907394596_0012/
18/09/25 21:45:50 INFO mapreduce.Job: Running job: job_1537907394596_0012
18/09/25 21:45:59 INFO mapreduce.Job: Job job_1537907394596_0012 running in uber mode : false
18/09/25 21:45:59 INFO mapreduce.Job:  map 0% reduce 0%
18/09

# Question 6: Document Classification Task Overview.
The week 2 assigned reading from Chapter 13 of _Introduction to Information Retrieval_ by Manning, Raghavan and Schutze provides a thorough introduction to the document classification task and the math behind Naive Bayes. In this question we'll use the example from Table 13.1 (reproduced below) to 'train' an unsmoothed Multinomial Naive Bayes model and classify a test document by hand.

<table>
<th>DocID</th>
<th>Class</th>
<th>Subject</th>
<th>Body</th>
<tr><td>Doc1</td><td>1</td><td></td><td>Chinese Beijing Chinese</td></tr>
<tr><td>Doc2</td><td>1</td><td></td><td>Chinese Chinese Shanghai</td></tr>
<tr><td>Doc3</td><td>1</td><td></td><td>Chinese Macao</td></tr>
<tr><td>Doc4</td><td>0</td><td></td><td>Tokyo Japan Chinese</td></tr>
</table>

### Q6 Tasks:
* __a) short response:__ Equation 13.3 in Manning, Raghavan and Shutze shows how a Multinomial Naive Bayes model classifies a document. It predicts the class, $c$, for which the estimated conditional probability of the class given the document's contents,  $\hat{P}(c|d)$, is greatest. In this equation what two pieces of information are required to calculate  $\hat{P}(c|d)$? Your answer should include both mathematical notatation and verbal explanation.


* __b) short response:__ The Enron data includes two classes of documents: `spam` and `ham` (they're actually labeled `1` and `0`). In plain English, explain what  $\hat{P}(c)$ and   $\hat{P}(t_{k} | c)$ mean in the context of this data. How will we would estimate these values from a training corpus? How many passes over the data would we need to make to retrieve this information for all classes and all words?


* __c) hand calculations:__ Above we've reproduced the document classification example from the textbook (we added an empty subject field to mimic the Enron data format). Remember that the classes in this "Chinese Example" are `1` (about China) and `0` (not about China). Calculate the class priors and the conditional probabilities for an __unsmoothed__ Multinomial Naive Bayes model trained on this data. Show the calculations that lead to your result using markdown and $\LaTeX$ in the space provided or by embedding an image of your hand written work. [`NOTE:` _Your results should NOT match those in the text -- they are training a model with +1 smoothing you are training a model without smoothing_]


* __d) hand calculations:__ Use the model you trained to classify the following test document: `Chinese Chinese Chinese Tokyo Japan`. Show the calculations that lead to your result using markdown and   $\LaTeX$ in the space provided or by embedding an image of your hand written work.


* __e) short response:__ Compare the classification you get from this unsmoothed model in `d`/`e` to the results in the textbook's "Example 1" which reflects a model with Laplace plus 1 smoothing. How does smoothing affect our inference?

### Q6 Student Answers:
> __a)__ To cacalute $\hat{P}(c|d)$, we would need to know 1) the prior $\hat{P}(c)$ which is the estimated probability of class c and 2) the $\hat{P}(t_k|c)$ which is the estimated probability of the occurence of term $t_k$ given the class c.

> __b)__ In this case, $\hat{P}(c)$ means the estimated probability of spam emails (if c denotes 'spam'). This can be calculated using (# of spam emails) / (total # of emails) from the training corpus. $\hat{P}(t_k|c)$ is the estimated probability of the occurence of term $t_k$ given the the email is a spam. This can be calculated using (# of occurences of term $t_k$ in spam) / (# of words in spam). This calculation only needs one pass through the training data. 

> __c)__ Priors: $\hat{P}(c)$ = $3/4$; $\hat{P}(\overline{c})$ = $1/4$

>Conditional Probabilities:

>>$\hat{P}(Chinese|c)$ = $5/8$

>>$\hat{P}(Beijing|c)$ = $\hat{P}(Shanghai|c)$= $\hat{P}(Macao|c)$= $1/8$

>>$\hat{P}(Tokyo|c)$ = $\hat{P}(Japan|c)$ = $0/8$ = $0$

>>$\hat{P}(Chinese|\overline{c})$ = $\hat{P}(Tokyo|\overline{c})$ = $\hat{P}(Japan|\overline{c})$ = $1/3$

>>$\hat{P}(Beijing|\overline{c})$ = $\hat{P}(Shanghai|\overline{c})$= $\hat{P}(Macao|\overline{c})$= $0/3$ = $0$



> __d)__ 

>$\hat{P}(c|d)$ = $$3/4 * (5/8)^3 * 0 * 0 = 0$$

>$\hat{P}(\overline{c}|d)$ = $$1/4 * (1/3)^3 * 1/3 * 1/3 = 0.001$$

> -> d is classified as $\overline{c}$


> __e)__ Our result is opposite from the example in the book with Laplace plus 1 smoothing. The smoothing helps with dealing with the conditional probablities of terms that don't occur in a specific class. Hence in this case, $\hat{P}(Tokyo|c)$ would have been a non-zero value with the smoothing


In [3]:
# part d/e - if you didn't write out your calcuations above, embed a picture of them here:
# from IPython.display import Image
# Image(filename="path-to-hand-calulations-image.png")

# Question 7: Naive Bayes Inference.
In the next two questions you'll write code to parallelize the Naive Bayes calculations that you performed above. We'll do this in two phases: one MapReduce job to perform training and a second MapReduce to perform inference. While in practice we'd need to train a model before we can use it to classify documents, for learning purposes we're going to develop our code in the opposite order. By first focusing on the pieces of information/format we need to perform the classification (inference) task you should find it easier to develop a solid implementation for training phase when you get to question 8 below. In both of these questions we'll continue to use the Chinese example corpus from the textbook to help us test our MapReduce code as we develop it. Below we've reproduced the corpus, test set and model in text format that matches the Enron data.

### Q7 Tasks:
* __a) short response:__ run the provided cells to create the example files and load them in to HDFS. Then take a closer look at __`NBmodel.txt`__. This text file represents a Naive Bayes model trained (with Laplace +1 smoothing) on the example corpus. What are the 'keys' and 'values' in this file? Which record means something slightly different than the rest? The value field of each record includes two numbers which will be helpful for debugging but which we don't actually need to perform inference -- what are they? [`HINT`: _This file represents the model from Example 13.1 in the textbook, if you're having trouble getting oriented try comparing our file to the numbers in that example._]


* __b) short response:__ When performing Naive Bayes in practice instead of multiplying the probabilities (as in equation 13.3) we add their logs (as in equation 13.4). Why do we choose to work with log probabilities? If we had an unsmoothed model, what potential error could arise from this transformation?


* __c) short response:__ Documents 6 and 8 in the test set include a word that did not appear in the training corpus (and as a result does not appear in the model). What should we do at inference time when we need a class conditional probability for this word?


* __d) short response:__ The goal of our MapReduce job is to stream over the test set and classify each document by peforming the calculation from equation 13.4. To do this we'll load the model file (which contains the probabilities for equation 13.4) into memory on the nodes where we do our mapping. This is called an in-memory join. Does loading a model 'state' like this depart from the functional programming principles? Explain why or why not. From a scability perspective when would this kind of memory use be justified? when would it be unwise?


* __e) code:__ Complete the code in __`NaiveBayes/classify_mapper.py`__. Read the docstring carefully to understand how this script should work and the format it should return. Run the provided unit tests to confirm that your script works as expected then write a Hadoop streaming job to classify the Chinese example test set. [`HINT 1:` _you shouldn't need a reducer for this one._ `HINT 2:` _Don't forget to add the model file to the_ `-files` _parameter in your Hadoop streaming job so that it gets shipped to the mapper nodes where it will be accessed by your script._]


* __f) short response:__ In our test example and in the Enron data set we have fairly short documents. Since these fit fine in memory on a mapper node we didn't need a reducer and could just do all of our calculations in the mapper. However with much longer documents (eg. books) we might want a higher level of parallelization -- for example we might want to process parts of a document on different nodes. In this hypothetical scenario how would our algorithm design change? What could the mappers still do? What key-value structure would they emit? What would the reducers have to do as a last step?

### Q7 Student Answers:
> __a)__ They keys are the word and the values are <number of occurrences in non-Chinese docs, number of occurrences in Chinese docs, conditional probability of word in non-Chinese docs, smoothed conditional smoothed probability of word in Chinese docs>. The absolute number of occurrences in non-Chinese/Chinese docs are helpful for debugging but not needed for the inference.

> __b)__ When many conditional probabilities are multiplied, it can result in a floating point underflow. This would not be a problem if we take log probabilities and sum them up. For an unsmoothed model, taking logarithm, however, can cause error if the word occurence in a certain class is 0. (log(0) is undefined)

> __c)__ If the word in the test set is not in the training set, we should just define the log of its conditional probability as 0 for inference since we don't have information on whether this word is predictive of either class.

> __d)__ I don't think it departs from the functional programming principle because it's a state that won't be changed and stays immutable throughout the streaming job. This is scalable as long as it's a relatively lightweight model that will be used by all/most processes. This would not be a good idea if it's a model that takes a lot of memory and won't serve many parallel processes.

> __e)__ Complete the coding portion of this question before answering 'f'.

> __f)__ If we use a reducer to aggregate the prior and conditional probabilities, we should not calculate the predicted class in the mapper anymore. Mappers can still be used to scan documents and calculate the partial sums of the conditional probabilities.They would emit doc id, partial sum of log(p(Ham) and log(p(Spam)) but it will be up to the reducers to sum up the log conditional probabilities and compute the predicted class then.


Run these cells to create the example corpus and model.

In [6]:
%%writefile NaiveBayes/chineseTrain.txt
D1	1		Chinese Beijing Chinese
D2	1		Chinese Chinese Shanghai
D3	1		Chinese Macao
D4	0		Tokyo Japan Chinese

Overwriting NaiveBayes/chineseTrain.txt


In [7]:
%%writefile NaiveBayes/chineseTest.txt
D5	1		Chinese Chinese Chinese Tokyo Japan
D6	1		Beijing Shanghai Trade
D7	0		Japan Macao Tokyo
D8	0		Tokyo Japan Trade

Overwriting NaiveBayes/chineseTest.txt


In [8]:
%%writefile NBmodel.txt
beijing	0.0,1.0,0.111111111111,0.142857142857
chinese	1.0,5.0,0.222222222222,0.428571428571
tokyo	1.0,0.0,0.222222222222,0.0714285714286
shanghai	0.0,1.0,0.111111111111,0.142857142857
ClassPriors	1.0,3.0,0.25,0.75
japan	1.0,0.0,0.222222222222,0.0714285714286
macao	0.0,1.0,0.111111111111,0.142857142857

Overwriting NBmodel.txt


In [7]:
# load the data files into HDFS
!hdfs dfs -copyFromLocal NaiveBayes/chineseTrain.txt {HDFS_DIR}
!hdfs dfs -copyFromLocal NaiveBayes/chineseTest.txt {HDFS_DIR}

Your work for `part e` starts here:

In [10]:
# part e - do your work in NaiveBayes/classify_mapper.py first, then run this cell.
!chmod a+x NaiveBayes/classify_mapper.py

In [12]:
# part e - unit test NaiveBayes/classify_mapper.py (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py | column -t

d5  1  -8.90668134500626   -8.10769031284611   1
d6  1  -5.780743515794329  -4.179502370564408  1
d7  0  -6.591673732011658  -7.511706880737812  0
d8  0  -4.394449154674438  -5.565796731681498  0


In [8]:
# part e - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/chinese-output

rm: `/user/root/HW2/chinese-output': No such file or directory


In [9]:
# part e - write your Hadooop streaming job here
!hadoop jar {JAR_FILE} \
  -files NBmodel.txt,NaiveBayes/classify_mapper.py \
  -mapper classify_mapper.py \
  -reducer /bin/cat \
  -input {HDFS_DIR}/chineseTest.txt \
  -output {HDFS_DIR}/chinese-output \
  -numReduceTasks 1 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.0.jar] /tmp/streamjob9201339597604586955.jar tmpDir=null
18/09/28 18:11:47 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/09/28 18:11:48 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/09/28 18:11:49 INFO mapred.FileInputFormat: Total input paths to process : 1
18/09/28 18:11:49 INFO mapreduce.JobSubmitter: number of splits:2
18/09/28 18:11:49 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1538158114445_0001
18/09/28 18:11:50 INFO impl.YarnClientImpl: Submitted application application_1538158114445_0001
18/09/28 18:11:50 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1538158114445_0001/
18/09/28 18:11:50 INFO mapreduce.Job: Running job: job_1538158114445_0001
18/09/28 18:11:59 INFO mapreduce.Job: Job job_1538158114445_0001 running in uber mode : false
18/09/28 18:11:59 INFO mapreduce.Job:  map 0% reduce 0%
18/09

In [10]:
# part e - retrieve test set results from HDFS (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/chinese-output/part-000* > NaiveBayes/chineseResults.txt

In [11]:
# part e - take a look (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseResults.txt | column -t

d5  1  -8.90668134500626   -8.10769031284611   1
d6  1  -5.780743515794329  -4.179502370564408  1
d7  0  -6.591673732011658  -7.511706880737812  0
d8  0  -4.394449154674438  -5.565796731681498  0


<table>
<th> Expected output for the test set:</th>
<tr align=Left><td><pre>
d5	1	-8.90668134	-8.10769031	1
d6	1	-5.78074351	-4.17950237	1
d7	0	-6.59167373	-7.51170688	0
d8	0	-4.39444915	-5.56579673	0
</pre></td><tr>
</table>

# Question 8: Naive Bayes Training.
In Question 7 we used a model that we had trained by hand. Next we'll develop the code to do that same training in parallel, making it suitable for use with larger corpora (like the Enron emails). The end result of the MapReduce job you write in this question should be a model text file that looks just like the example (`NBmodel.txt`) that we created by hand above.

To refresh your memory about the training process take a look at  `6a` and `6b` where you described the pieces of information you'll need to collect in order to encode a Multinomial Naive Bayes model. We now want to retrieve those pieces of information while streaming over a corpus. The bulk of the task will be very similar to the word counting excercises you've already done but you may want to consider a slightly different key-value record structure to efficiently tally counts for each class. 

The most challenging (interesting?) design question will be how to retrieve the totals (# of documents and # of words in documents for each class). Of course, counting these numbers is easy. The hard part is the timing: you'll need to make sure you have the counts totalled up _before_ you start estimating the class conditional probabilities for each word. It would be best (i.e. most scalable) if we could find a way to do this tallying without storing the whole vocabulary in memory... Use an appropriate MapReduce design pattern to implement this efficiently! 

__`IMPORTANT NOTE:`__ Please use a single reducer for all of the jobs in this question.

### Q8 Tasks:
* __a) make a plan:__  Fill in the docstrings for __`NaiveBayes/train_mapper.py`__ and __`NaiveBayes/train_reducer.py`__ to appropriately reflect the format that each script will input/output. [`HINT:` _the input files_ (`enronemail_1h.txt` & `chineseTrain.txt`) _have a prespecified format and your output file should match_ `NBmodel.txt` _so you really only have to decide on an internal format for Hadoop_].


* __b) implement it:__ Complete the code in __`NaiveBayes/train_mapper.py`__ and __`NaiveBayes/train_reducer.py`__ so that together they train a Multinomial Naive Bayes model __with no smoothing__. Make sure your end result is formatted correctly (see note above). Test your scripts independently and together (using `chineseTrain.txt` or test input of your own devising). When you are satisfied with your Python code design and run a Hadoop streaming command to run your job in parallel on the __chineseTrain.txt__. Confirm that your trained model matches your hand calculations from Question 5.


* __c) short response:__ We saw in Question 6 that adding Laplace +1 smoothing makes our classifications less sensitve to rare words. However implementing this technique requires access to one additional piece of information that we had not previously used in our Naive Bayes training. What is that extra piece of information? [`HINT:` see equation 13.7 in Manning, Raghavan and Schutze].


* __d) short response:__ There are three approaches that we could take to handle the extra piece of information you identified in `c`: 1) we could hard code it into our reducer (_where would we get it in the first place?_). Or 2) we could compute it inside the reducer which would require storing some information in memory (_what information?_). Or 3) we could compute it in the reducer without storing any bulky information in memory but then we'd need some postprocessing or a second MapReduce job to complete the calculation (_why?_). Breifly explain what is non-ideal about each of these options. BONUS: which of these options is incompatible with using multiple reducers?


* __e) code + short response:__ Choose one of the 3 options above. State your choice & reasoning in the space below then use that strategy to complete the code in __`NaiveBayes/train_reducer_smooth.py`__. Test this alternate reducer then write and run a Hadoop streaming job to train an MNB model with smoothing on the Chinese example. Your results should match the model that we provided for you above (and the calculations in the textbook example). [`HINT:` _don't start from scratch with this one -- you can just copy over your reducer code from part `b` and make the needed modifications_].

### Q8 Student Answers:
> __ c)__ We will also need information on the number of unique words in the vocabulary

> __ d)__ 

> 1) We can find out the total vocabulary from scanning all the words in the corpus using a deduped sequense like "set" and hardcode the number into the reducer. This is not ideal because 1) this might take a lot of computation power when we have a large corpus and 2) everytime when running a new training set, a new variable needs to be hardcoded.

> 2) We could use a dictionary in the reducer to store words and word count in memory and just look at how many keys the dictionary end up having in the end to get the vocabulary size before any further calculation. But this is not scalable since we are storing in-memory and theoretically with a large enough corpus, it will crach the machine. This also would not work if we use multiple reducers since each reducer would only see a limited set of words

> 3) We could implement a counter in the reducer which counts the number of total vocabulary seen. This is scalable since it's not storing anything but a counter in the memory. However, this will need post processing/second MapReduce Job since the count won't be finalized until all the words have been run through.

> __ e)__ I chose option 2 to store words and counts in memory during the reducer phase. I chose it because of the size of corpus that we are dealing with in this context and having an in-memory storage makes the process succinct and autonomous. Although, this would not be a good design for a very large dataset. In that case, I would go with 3), use one reducer to spit out <word, true class, count> that includes a word called "!vocab size" which is a built-in counter that counts distinct word. And then make sure to feed "!vocab size" into the second reducer to do the calculation 


In [50]:
# part a - do your work in train_mapper.py and train_reducer.py then RUN THIS CELL AS IS
!chmod a+x NaiveBayes/train_mapper.py
!chmod a+x NaiveBayes/train_reducer.py
!echo "=========== MAPPER DOCSTRING ============"
!head -n 8 NaiveBayes/train_mapper.py | tail -n 6
!echo "=========== REDUCER DOCSTRING ============"
!head -n 8 NaiveBayes/train_reducer.py | tail -n 6

=========== MAPPER DOCSTRING ============
Mapper reads in text documents and emits word counts by class.
    
INPUT:
    ID \t true_class \t subject \t body \n
OUTPUT:
    word \t true_class \t count
=========== REDUCER DOCSTRING ============
Reducer aggregates word counts by class and emits frequencies and conditional probabilities.
    
INPUT:
    word \t true_class \t count
OUTPUT:
    word \t cntHam \t cntSpam \t cpHam \t cpSpam


__`part b starts here`:__ MNB _without_ Smoothing (training on Chinese Example Corpus).

In [41]:
# part b - write a unit test for your mapper here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py | sort -k1,1 | column -t

chinese     1  1
beijing     1  1
chinese     1  1
chinese     1  1
chinese     1  1
shanghai    1  1
chinese     1  1
macao       1  1
tokyo       0  1
japan       0  1
chinese     0  1
!wordcount  0  3
!wordcount  1  8
!!doccount  0  1
!!doccount  1  3


In [126]:
# part b - write a unit test for your reducer here
#write succesfully run mapper output as a test input file
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py | sort -k1,1 | column -t > NaiveBayes/train_reducer_test_input.py

#test reducer
!cat NaiveBayes/train_reducer_test_input.py | NaiveBayes/train_reducer.py | column -t

beijing      0,1,0.0,0.125
chinese      1,5,0.3333333333333333,0.625
japan        1,0,0.3333333333333333,0.0
macao        0,1,0.0,0.125
shanghai     0,1,0.0,0.125
tokyo        1,0,0.3333333333333333,0.0
ClassPriors  1,3,0.25,0.75


In [127]:
# part b - write a systems test for your mapper + reducer together here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py | sort -k1,1 | NaiveBayes/train_reducer.py | column -t

beijing      0,1,0.0,0.125
chinese      1,5,0.3333333333333333,0.625
japan        1,0,0.3333333333333333,0.0
macao        0,1,0.0,0.125
shanghai     0,1,0.0,0.125
tokyo        1,0,0.3333333333333333,0.0
ClassPriors  1,3,0.25,0.75


In [128]:
# part b - clear (and name) an output directory in HDFS for your unsmoothed chinese NB model
!hdfs dfs -rm -r {HDFS_DIR}/unsmooth-model-output

Deleted /user/root/HW2/unsmooth-model-output


In [129]:
# part b - write your hadoop streaming job
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer.py \
  -mapper train_mapper.py \
  -reducer train_reducer.py \
  -input {HDFS_DIR}/chineseTrain.txt \
  -output {HDFS_DIR}/unsmooth-model-output \
  -numReduceTasks 1 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.0.jar] /tmp/streamjob3302524331115068054.jar tmpDir=null
18/10/01 04:40:41 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/10/01 04:40:41 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/10/01 04:40:42 INFO mapred.FileInputFormat: Total input paths to process : 1
18/10/01 04:40:42 INFO mapreduce.JobSubmitter: number of splits:2
18/10/01 04:40:42 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1538158114445_0017
18/10/01 04:40:42 INFO impl.YarnClientImpl: Submitted application application_1538158114445_0017
18/10/01 04:40:42 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1538158114445_0017/
18/10/01 04:40:42 INFO mapreduce.Job: Running job: job_1538158114445_0017
18/10/01 04:40:49 INFO mapreduce.Job: Job job_1538158114445_0017 running in uber mode : false
18/10/01 04:40:49 INFO mapreduce.Job:  map 0% reduce 0%
18/10

In [130]:
# part b - extract your results (i.e. model) to a local file
!hdfs dfs -cat {HDFS_DIR}/unsmooth-model-output/part-000* > NaiveBayes/UnsmoothModelResults.txt

In [131]:
# part b - print your model so that we can confirm that it matches expected results
!cat NaiveBayes/UnsmoothModelResults.txt | column -t

beijing      0,1,0.0,0.125
chinese      1,5,0.3333333333333333,0.625
japan        1,0,0.3333333333333333,0.0
macao        0,1,0.0,0.125
shanghai     0,1,0.0,0.125
tokyo        1,0,0.3333333333333333,0.0
ClassPriors  1,3,0.25,0.75


__`part e starts here`:__ MNB _with_ Smoothing (training on Chinese Example Corpus).

In [134]:
# part e - write a unit test for your NEW reducer here
!chmod a+x NaiveBayes/train_reducer_smooth.py
!cat NaiveBayes/train_reducer_test_input.py | NaiveBayes/train_reducer_smooth.py | column -t

tokyo        1,0,0.2222222222222222,0.07142857142857142
japan        1,0,0.2222222222222222,0.07142857142857142
beijing      0,1,0.1111111111111111,0.14285714285714285
macao        0,1,0.1111111111111111,0.14285714285714285
chinese      1,5,0.2222222222222222,0.42857142857142855
shanghai     0,1,0.1111111111111111,0.14285714285714285
ClassPriors  1,3,0.25,0.75


In [135]:
# part e - write a systems test for your mapper + reducer together here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py | sort -k1,1 | NaiveBayes/train_reducer_smooth.py | column -t

shanghai     0,1,0.1111111111111111,0.14285714285714285
chinese      1,5,0.2222222222222222,0.42857142857142855
tokyo        1,0,0.2222222222222222,0.07142857142857142
beijing      0,1,0.1111111111111111,0.14285714285714285
macao        0,1,0.1111111111111111,0.14285714285714285
japan        1,0,0.2222222222222222,0.07142857142857142
ClassPriors  1,3,0.25,0.75


In [136]:
# part e - clear (and name) an output directory in HDFS for your SMOOTHED chinese NB model
!hdfs dfs -rm -r {HDFS_DIR}/smooth-model-output

Deleted /user/root/HW2/smooth-model-output


In [137]:
# part e - write your hadoop streaming job
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer_smooth.py \
  -mapper train_mapper.py \
  -reducer train_reducer_smooth.py \
  -input {HDFS_DIR}/chineseTrain.txt \
  -output {HDFS_DIR}/smooth-model-output \
  -numReduceTasks 1 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.0.jar] /tmp/streamjob2514798626294337002.jar tmpDir=null
18/10/01 04:42:33 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/10/01 04:42:33 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/10/01 04:42:35 INFO mapred.FileInputFormat: Total input paths to process : 1
18/10/01 04:42:35 INFO mapreduce.JobSubmitter: number of splits:2
18/10/01 04:42:35 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1538158114445_0018
18/10/01 04:42:36 INFO impl.YarnClientImpl: Submitted application application_1538158114445_0018
18/10/01 04:42:36 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1538158114445_0018/
18/10/01 04:42:36 INFO mapreduce.Job: Running job: job_1538158114445_0018
18/10/01 04:42:43 INFO mapreduce.Job: Job job_1538158114445_0018 running in uber mode : false
18/10/01 04:42:43 INFO mapreduce.Job:  map 0% reduce 0%
18/10

In [138]:
# part e - extract your results (i.e. model) to a local file
!hdfs dfs -cat {HDFS_DIR}/smooth-model-output/part-000* > NaiveBayes/SmoothModelResults.txt

In [139]:
!cat NaiveBayes/SmoothModelResults.txt | column -t

japan        1,0,0.2222222222222222,0.07142857142857142
tokyo        1,0,0.2222222222222222,0.07142857142857142
chinese      1,5,0.2222222222222222,0.42857142857142855
beijing      0,1,0.1111111111111111,0.14285714285714285
macao        0,1,0.1111111111111111,0.14285714285714285
shanghai     0,1,0.1111111111111111,0.14285714285714285
ClassPriors  1,3,0.25,0.75


# Question 9: Enron Ham/Spam NB Classifier & Results.

Fantastic work. We're finally ready to perform Spam Classification on the Enron Corpus. In this question you'll run the analysis you've developed, report its performance, and draw some conclusions.

### Q9 Tasks:
* __a) train/test split:__ Run the provided code to split our Enron file into a training set and testing set then load them into HDFS. 

[`NOTE:` _If you hard coded the vocab size in question 8d make sure you re calculate the vocab size for just the training set!_]

* __b) train 2 models:__ Write Hadoop Streaming jobs to train MNB Models on the training set with and without smoothing. Save your models to local files at __`NaiveBayes/Unsmoothed/NBmodel.txt`__ and __`NaiveBayes/Smoothed/NBmodel.txt`__. [`NOTE:` _This naming is important because we wrote our classification task so that it expects a file of that name... if this inelegance frustrates you there is an alternative that would involve a few adjustments to your code [read more about it here](http://www.tnoda.com/blog/2013-11-23)._] Finally run the checks that we provide to confirm that your results are correct.


* __c) code:__ Recall that we designed our classification job with just a mapper. An efficient way to report the performance of our models would be to simply add a reducer phase to this job and compute precision and recall right there. Complete the code in __`NaiveBayes/evaluation_reducer.py`__ and then write Hadoop jobs to evaluate your two models on the test set. Report their performance side by side. [`NOTE:` if you need a refresher on precision, recall and F1-score [Wikipedia](https://en.wikipedia.org/wiki/F1_score) is a good resource.]


* __d) short response:__ Compare the performance of your two models. What do you notice about the unsmoothed model's predictions? Can you guess why this is happening? Which evaluation measure do you think is most relevant in our use case? [`NOTE:` _Feel free to answer using your common sense but if you want more information on evaluating the classification task checkout_ [this blogpost](https://tryolabs.com/blog/2013/03/25/why-accuracy-alone-bad-measure-classification-tasks-and-what-we-can-do-about-it/
) or [this paper](http://www.flinders.edu.au/science_engineering/fms/School-CSEM/publications/tech_reps-research_artfcts/TRRA_2007.pdf
)]


* __e) code + short response:__ Let's look at the top ten words with the highest conditional probability in `Spam` and in `Ham`. We'll do this by writing a Hadoop job that sorts the model file (`NaiveBayes/Smoothed/NBmodel.py`). Normally we'd have to run two jobs -- one that sorts on $P(word|ham)$ and another that sorts on $P(word|spam)$. However if we slighly modify the data format in the model file then we can get the top words in each class with just one job. We've written a mapper that will do just this for you. Read through __`NaiveBayes/model_sort_mapper.py`__ and then briefly explain how this mapper will allow us to partition and sort our model file. Write a Hadoop job that uses our mapper and `/bin/cat` for a reducer to partition and sort. Print out the top 10 words in each class (where 'top' == highest conditional probability).[`HINT:` _this should remind you a lot of what we did in Question 6._]


* __f) short response:__ What do you notice about the 'top words' we printed in `e`? How would increasing the smoothing parameter 'k' affect the probabilities for the top words that you identified for 'e'. How would they affect the probabilities of words that occur much more in one class than another? In summary, how does the smoothing parameter 'k' affect the bias and the variance of our model. [`NOTE:` _you do not need to code anything for this task, but if you are struggling with it you could try changing 'k' and see what happens to the test set. We don't recommend doing this exploration with the Enron data because it will be harder to see the impact with such a big vocabulary_]

### Q9 Student Answers:
> __d)__ The unsmoothed model has a lot of false negatives. This is likely due to the fact that any unseen words in the given class will have a conditional probability of 0. And in the model they will be defined as *-inf* when taking the log. With that, a document will be classified as negative even if it contains many words that have a high conditional probability of being positive, as long as it contains one or more words that were not seen in the postive class in the training set.

> __e)__ The mapper looks into each word and identifies the higher conditional probability and the class that probability belongs to. This allows the outputs to have a neat format with the class, and the higher conditional probability summarized as separate fields. We can then partition the results using the class and sort the results using the conditional probabilities.

> __f)__ The top 10 words in this case seem like just common English words with little evidence to a human reader that they would be the key difference maker in the classfication of ham vs. spam. By increasing k, the conditional probability for words that don't occur that often will have a more notable increase than these common words. An increased K would have little effect on these top words since they occur so commonly. 


> For those words which occur much more in one class than another, an increased k would decrease the difference between $\hat{P}(t_k|c)$ and $\hat{P}(t_k|\overline{c})$. 

> In summary, an increased smoothing would increase bias and decrease variance since the training fit would be more generalizable.

__Test/Train split__

In [77]:
# part a - test/train split (RUN THIS CELL AS IS)
!head -n 80 data/enronemail_1h.txt > data/enron_train.txt
!tail -n 20 data/enronemail_1h.txt > data/enron_test.txt
!hdfs dfs -copyFromLocal data/enron_train.txt {HDFS_DIR}
!hdfs dfs -copyFromLocal data/enron_test.txt {HDFS_DIR}

__Training__ (Enron MNB Model _without smoothing_ )

In [140]:
# part b -  Unsmoothed model (FILL IN THE MISSING CODE BELOW)

# clear the output directory
!hdfs dfs -rm -r {HDFS_DIR}/enron-model

# hadoop command
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer.py \
  -mapper train_mapper.py \
  -reducer train_reducer.py \
  -input {HDFS_DIR}/enron_train.txt \
  -output {HDFS_DIR}/enron-model \
  -numReduceTasks 1 \
  -cmdenv PATH={PATH}

# save the model locally
# !mkdir NaiveBayes/Unsmoothed
!hdfs dfs -cat {HDFS_DIR}/enron-model/part-000* > NaiveBayes/Unsmoothed/NBmodel.txt

Deleted /user/root/HW2/enron-model
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.0.jar] /tmp/streamjob1802214552539149139.jar tmpDir=null
18/10/01 04:43:33 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/10/01 04:43:33 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/10/01 04:43:34 INFO mapred.FileInputFormat: Total input paths to process : 1
18/10/01 04:43:34 INFO mapreduce.JobSubmitter: number of splits:2
18/10/01 04:43:35 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1538158114445_0019
18/10/01 04:43:35 INFO impl.YarnClientImpl: Submitted application application_1538158114445_0019
18/10/01 04:43:35 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1538158114445_0019/
18/10/01 04:43:35 INFO mapreduce.Job: Running job: job_1538158114445_0019
18/10/01 04:43:41 INFO mapreduce.Job: Job job_1538158114445_0019 running in uber mode : false
18/10/01 04:43:41 INFO map

In [141]:
# part b - check your UNSMOOTHED model results (RUN THIS CELL AS IS)
!grep assistance NaiveBayes/Unsmoothed/NBmodel.txt
# EXPECTED OUTPUT: assistance	2,4,0.000172547666293,0.000296823983378

assistance	2,4,0.0001725476662928134,0.00029682398337785694


In [142]:
# part b - check your UNSMOOTHED model results (RUN THIS CELL AS IS)
!grep money NaiveBayes/Unsmoothed/NBmodel.txt
# EXPECTED OUTPUT: money	1,22,8.62738331464e-05,0.00163253190858

money	1,22,8.62738331464067e-05,0.001632531908578213


__Training__ (Enron MNB Model _with Laplace +1 smoothing_ )

In [143]:
# part b -  Smoothed model (FILL IN THE MISSING CODE BELOW)

# clear the output directory
!hdfs dfs -rm -r {HDFS_DIR}/smooth-model

# hadoop command
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer_smooth.py \
  -mapper train_mapper.py \
  -reducer train_reducer_smooth.py \
  -input {HDFS_DIR}/enron_train.txt \
  -output {HDFS_DIR}/smooth-model \
  -numReduceTasks 1 \
  -cmdenv PATH={PATH}

# save the model locally
# !mkdir NaiveBayes/Smoothed
!hdfs dfs -cat {HDFS_DIR}/smooth-model/part-000* > NaiveBayes/Smoothed/NBmodel.txt

Deleted /user/root/HW2/smooth-model
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.0.jar] /tmp/streamjob8679407997338526006.jar tmpDir=null
18/10/01 04:44:32 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/10/01 04:44:32 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/10/01 04:44:33 INFO mapred.FileInputFormat: Total input paths to process : 1
18/10/01 04:44:33 INFO mapreduce.JobSubmitter: number of splits:2
18/10/01 04:44:34 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1538158114445_0020
18/10/01 04:44:34 INFO impl.YarnClientImpl: Submitted application application_1538158114445_0020
18/10/01 04:44:34 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1538158114445_0020/
18/10/01 04:44:34 INFO mapreduce.Job: Running job: job_1538158114445_0020
18/10/01 04:44:40 INFO mapreduce.Job: Job job_1538158114445_0020 running in uber mode : false
18/10/01 04:44:40 INFO ma

In [144]:
# part b - check your SMOOTHED model results (RUN THIS CELL AS IS)
!grep assistance NaiveBayes/Smoothed/NBmodel.txt
# EXPECTED OUTPUT: assistance	2,4,0.000185804533631,0.000277300205202

assistance	2,4,0.0001858045336306206,0.00027730020520215184


In [145]:
# part b - check your SMOOTHED model results (RUN THIS CELL AS IS)
!grep money NaiveBayes/Smoothed/NBmodel.txt
# EXPECTED OUTPUT: money	1,22,0.000123869689087,0.00127558094393

money	1,22,0.0001238696890870804,0.0012755809439298986


__Evaluation__

In [146]:
# part c - write your code in NaiveBayes/evaluation_reducer.py then RUN THIS
!chmod a+x NaiveBayes/evaluation_reducer.py

In [157]:
# part c - unit test your evaluation job on the chinese model (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py 
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py | NaiveBayes/evaluation_reducer.py | column -t

d5	1	-8.90668134500626	-8.10769031284611	1
d6	1	-5.780743515794329	-4.179502370564408	1
d7	0	-6.591673732011658	-7.511706880737812	0
d8	0	-4.394449154674438	-5.565796731681498	0
d5          1            -8.90668134500626   -8.10769031284611   True
d6          1            -5.780743515794329  -4.179502370564408  True
d7          0            -6.591673732011658  -7.511706880737812  True
d8          0            -4.394449154674438  -5.565796731681498  True
#           Documents:   4
True        Postitives:  2.0
True        Negatives:   2.0
False       Postitives:  0.0
False       Negatives:   0.0
Accuracy:   1.0
Precision:  1.0
Recall:     1.0
F-score:    1.0


In [158]:
# part c - Evaluate the UNSMOOTHED Model Here (FILL IN THE MISSING CODE)

# clear output directory
!hdfs dfs -rm -r {HDFS_DIR}/enron-model-eval-unsmoothed

# hadoop # hadoop job
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/classify_mapper.py,NaiveBayes/evaluation_reducer.py,NaiveBayes/Unsmoothed/NBmodel.txt \
  -mapper classify_mapper.py \
  -reducer evaluation_reducer.py \
  -input {HDFS_DIR}/enron_test.txt \
  -output {HDFS_DIR}/enron-model-eval-unsmoothed \
  -numReduceTasks 1 \
  -cmdenv PATH={PATH}

# retrieve results locally
!hdfs dfs -cat {HDFS_DIR}/enron-model-eval-unsmoothed/part-000* > NaiveBayes/Unsmoothed/results.txt

Deleted /user/root/HW2/enron-model-eval-unsmoothed
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.0.jar] /tmp/streamjob4680937568387105426.jar tmpDir=null
18/10/01 04:59:54 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/10/01 04:59:54 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/10/01 04:59:55 INFO mapred.FileInputFormat: Total input paths to process : 1
18/10/01 04:59:55 INFO mapreduce.JobSubmitter: number of splits:2
18/10/01 04:59:55 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1538158114445_0023
18/10/01 04:59:56 INFO impl.YarnClientImpl: Submitted application application_1538158114445_0023
18/10/01 04:59:56 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1538158114445_0023/
18/10/01 04:59:56 INFO mapreduce.Job: Running job: job_1538158114445_0023
18/10/01 05:00:02 INFO mapreduce.Job: Job job_1538158114445_0023 running in uber mode : false
18/10/01 0

In [159]:
# part c - Evaluate the SMOOTHED Model Here (FILL IN THE MISSING CODE)

# clear output directory
!hdfs dfs -rm -r {HDFS_DIR}/enron-model-eval-smoothed

# hadoop # hadoop job
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/classify_mapper.py,NaiveBayes/evaluation_reducer.py,NaiveBayes/Smoothed/NBmodel.txt \
  -mapper classify_mapper.py \
  -reducer evaluation_reducer.py \
  -input {HDFS_DIR}/enron_test.txt \
  -output {HDFS_DIR}/enron-model-eval-smoothed \
  -numReduceTasks 1 \
  -cmdenv PATH={PATH}

# retrieve results locally
!hdfs dfs -cat {HDFS_DIR}/enron-model-eval-smoothed/part-000* > NaiveBayes/Smoothed/results.txt

Deleted /user/root/HW2/enron-model-eval-smoothed
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.0.jar] /tmp/streamjob6813485822985869581.jar tmpDir=null
18/10/01 05:00:23 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/10/01 05:00:23 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/10/01 05:00:24 INFO mapred.FileInputFormat: Total input paths to process : 1
18/10/01 05:00:24 INFO mapreduce.JobSubmitter: number of splits:2
18/10/01 05:00:24 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1538158114445_0024
18/10/01 05:00:24 INFO impl.YarnClientImpl: Submitted application application_1538158114445_0024
18/10/01 05:00:24 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1538158114445_0024/
18/10/01 05:00:24 INFO mapreduce.Job: Running job: job_1538158114445_0024
18/10/01 05:00:31 INFO mapreduce.Job: Job job_1538158114445_0024 running in uber mode : false
18/10/01 05:

In [160]:
# part c - display results 
# NOTE: feel free to modify the tail commands to match the format of your results file
print('=========== UNSMOOTHED MODEL ============')
!tail -n 9 NaiveBayes/Unsmoothed/results.txt
print('=========== SMOOTHED MODEL ============')
!tail -n 9 NaiveBayes/Smoothed/results.txt

=========== UNSMOOTHED MODEL ============
# Documents:	20	
True Postitives:	1.0	
True Negatives:	9.0	
False Postitives:	0.0	
False Negatives:	10.0	
Accuracy:	0.5	
Precision:	1.0	
Recall:	0.09090909090909091	
F-score:	0.16666666666666669	
=========== SMOOTHED MODEL ============
# Documents:	20	
True Postitives:	11.0	
True Negatives:	6.0	
False Postitives:	3.0	
False Negatives:	0.0	
Accuracy:	0.85	
Precision:	0.7857142857142857	
Recall:	1.0	
F-score:	0.88	


__`EXPECTED RESULTS:`__ 
<table>
<th>Unsmoothed Model</th>
<th>Smoothed Model</th>
<tr>
<td><pre>
# Documents:	20
True Positives:	1
True Negatives:	9
False Positives:	0
False Negatives:	10
Accuracy	0.5
Precision	1.0
Recall	0.0909
F-Score	0.1666
</pre></td>
<td><pre>
# Documents:	20
True Positives:	11
True Negatives:	6
False Positives:	3
False Negatives:	0
Accuracy	0.85
Precision	0.7857
Recall	1.0
F-Score	0.88
</pre></td>
</tr>
</table>

__`NOTE:`__ _Don't be too disappointed if these seem low to you. We've trained and tested on a very very small corpus... bigger datasets coming soon!_

__`part e starts here:`__

In [172]:
!chmod a+x NaiveBayes/model_sort_mapper.py
# !cat NaiveBayes/Smoothed/NBmodel.txt | NaiveBayes/model_sort_mapper.py | sort -k4nr,4

In [175]:
# part e - write your Hadoop job here (sort smoothed model on P(word|class))

# clear output directory
!hdfs dfs -rm -r {HDFS_DIR}/top_10_each_class

# hadoop job
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=4 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k4nr,4" \
  -D mapreduce.partition.keypartitioner.options="-k3,3" \
  -files NaiveBayes/model_sort_mapper.py \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -mapper model_sort_mapper.py \
  -reducer /bin/cat \
  -input {HDFS_DIR}/smooth-model/part-000* \
  -output {HDFS_DIR}/top_10_each_class \
  -cmdenv PATH={PATH}\
  -numReduceTasks 2



Deleted /user/root/HW2/top_10_each_class
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.0.jar] /tmp/streamjob5349364496256530330.jar tmpDir=null
18/10/01 17:39:48 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/10/01 17:39:48 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/10/01 17:39:49 INFO mapred.FileInputFormat: Total input paths to process : 1
18/10/01 17:39:49 INFO mapreduce.JobSubmitter: number of splits:2
18/10/01 17:39:49 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1538158114445_0032
18/10/01 17:39:49 INFO impl.YarnClientImpl: Submitted application application_1538158114445_0032
18/10/01 17:39:49 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1538158114445_0032/
18/10/01 17:39:49 INFO mapreduce.Job: Running job: job_1538158114445_0032
18/10/01 17:39:55 INFO mapreduce.Job: Job job_1538158114445_0032 running in uber mode : false
18/10/01 17:39:55 IN

In [176]:
# part e - print top words in each class
for p in range(2):
    print('='*10,f'PARTITION{p+1}','='*10)
    !hdfs dfs -cat {HDFS_DIR}/top_10_each_class/part-0000{p} | head -n 10

========== PARTITION1 ==========
ClassPriors	47,33,0.5875,0.4125	ham	0.5875	
ect	378,0,0.023473306082001735,5.546004104043037e-05	ham	0.023473306082001735	
and	258,277,0.01604112473677691,0.015417891409239643	ham	0.01604112473677691	
hou	203,0,0.0126347082868822,5.546004104043037e-05	ham	0.0126347082868822	
in	160,157,0.009971509971509971,0.008762686484387999	ham	0.009971509971509971	
for	148,153,0.00922829183698749,0.008540846320226277	ham	0.00922829183698749	
on	122,95,0.007617985878855444,0.005324163939881316	ham	0.007617985878855444	
enron	116,0,0.007246376811594203,5.546004104043037e-05	ham	0.007246376811594203	
will	113,69,0.007060572277963582,0.003882202872830126	ham	0.007060572277963582	
i	113,106,0.007060572277963582,0.00593422439132605	ham	0.007060572277963582	
cat: Unable to write to output stream.
========== PARTITION2 ==========
the	453,535,0.02811841942276725,0.029726581997670677	spam	0.029726581997670677	
to	350,420,0.021739130434782608,0.023348677278021184	spam	0.023348

# HW2 ends here, please refer to the `README.md` for submission instructions.